## Producer S_Topic 

In [1]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


# option 1 : Read all at once 

In [ ]:
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
import time
import random
import numpy as np
import json  


if __name__ == "__main__":
    print("Kafka Producer Application Started ... ")
    
    producer = KafkaProducer(
    bootstrap_servers='185.185.126.143:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    
    #small data set 
    filepath = "/home/jovyan/work/notebooks/Yelp_20171201.json"
    #big data set 
    #filepath = "/home/jovyan/work/notebooks/Yelp_2018.json"
    
    yelp_df = pd.read_json(filepath , lines=True)
    
    print(yelp_df)
  
    yelp_df['order_id'] = np.arange(len(yelp_df))    ##what is this , field ?
    yelp_list = yelp_df.to_dict(orient="records")

In [ ]:
    message_list = []
    message = None
    for message in yelp_list:
        
        message_fields_value_list = []
               
        message_fields_value_list.append(message["review_id"])
        message_fields_value_list.append(message["business_id"])
        message_fields_value_list.append(message["user_id"])
        message_fields_value_list.append(message["stars"])
        message_fields_value_list.append(message["Date_Only"])
        message_fields_value_list.append(message["Time_Only"])
        message_fields_value_list.append(message["Date_Hour"])
        message_fields_value_list.append(message["Date_Minute"])
        message_fields_value_list.append(message["DateTime"])
        #message_fields_value_list.append(message["text"])   #must be last because of comma issues

        message = ','.join(str(v) for v in message_fields_value_list)
        print("Message Type: ", type(message))
        print("Message: ", message)
        producer.send("S_Topic", message)         #Just one message , how can we make this more sustainable -not 3 lines?
        #producer.send("S_Topic", message)
        #producer.send("Topic", message)
        print("Start : %s" % time.ctime())
        time.sleep(1)                          ##Time Control x messages per y minutes 
        print("End : %s" % time.ctime())


    print("Kafka Producer Application Completed. ")

# option 2: Read in line by line with spark to not clog memory 

In [2]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('json-changes-event-consumer')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,mysql:mysql-connector-java:8.0.11,com.datastax.spark:spark-cassandra-connector_2.12:3.0.1")
         .config("spark.cassandra.connection.host","185.185.126.143")
         .config("spark.cassandra.connection.port","9742")
         .config("spark.cassandra.auth.username","cassandra")
         .config("spark.cassandra.auth.password","cassandra")
         .getOrCreate())

sc = spark.sparkContext

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
mysql#mysql-connector-java added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5dd5137d-e7a1-44e6-914f-8b7457fc8d8e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found mysql#mysql-connector-java;8.0.11 in central
	found com.google.p

In [3]:
#df = spark.read.json('Yelp_20171201.json')
df = spark.read.json('Yelp_2018.json')

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

for i in range(10) :
    df.select(to_json(struct("review_id" ,"business_id" , "user_id" , "stars", "Date_Only", "Time_Only" , "Date_Hour", "Date_Minute", "DateTime")).alias("value")) \
    .write.mode('append') \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "185.185.126.143:9092") \
    .option("topic", "S2_TOPIC") \
    .save()
    print('producing data batch -- '+ str(i))
    time.sleep(60)


producing data batch -- 0


producing data batch -- 1


producing data batch -- 2


producing data batch -- 3


producing data batch -- 4


producing data batch -- 5


producing data batch -- 6


producing data batch -- 7


producing data batch -- 8


producing data batch -- 9


In [5]:
df.count()

1084335